#### Feature Engineering  

Dataset: 
- _calls_clean.csv_
- _internet_clean.csv_
- _messages_clean.csv_
- _plans_clean.csv_
- _users_clean.csv_

Author: Luis Sergio Pastrana Lemus  
Date: 2025-05-14

# Feature engineering – Purchasing Activity Dataset

## __1. Libraries__

In [1]:
from IPython.display import display, HTML
from functools import partial
import numpy as np
import os
import pandas as pd
from pathlib import Path
import sys

# Define project root dynamically, gets the current directory from whick the notebook belongs and moves one level upper
project_root = Path.cwd().parent

# Add src to sys.path if it is not already
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import function directly (more controlled than import *)
from src import *

## __2. Path to Data file__

In [2]:
# Build route to data file and upload
data_file_path = project_root / "data" / "processed" / "clean"
df_calls = load_dataset_from_csv(data_file_path, "calls_clean.csv", sep=',', header='infer')
df_internet = load_dataset_from_csv(data_file_path, "internet_clean.csv", sep=',', header='infer')
df_messages = load_dataset_from_csv(data_file_path, "messages_clean.csv", sep=',', header='infer')
df_plans = load_dataset_from_csv(data_file_path, "plans_clean.csv", sep=',', header='infer')
df_users = load_dataset_from_csv(data_file_path, "users_clean.csv", sep=',', header='infer')


In [3]:
# Format notebook output
format_notebook()

## 3 __Functions__

In [4]:
# Function for calculating the revenue per user
def revenue(row, revenue_type, column):
    # Plan parameters
    plans = {
        'surf': {
            'base_fee': 20, 'minutes_limit': 500, 'messages_limit': 50, 'mb_limit': 15360,
            'minute_over': 0.03, 'message_over': 0.03, 'mb_over': 0.01
        },
        'ultimate': {
            'base_fee': 70, 'minutes_limit': 3000, 'messages_limit': 1000, 'mb_limit': 30720,
            'minute_over': 0.01, 'message_over': 0.01, 'mb_over': 0.007
        }
    }

    # Basic validations
    valid_types = {'total', 'duration', 'messages', 'traffic'}
    if revenue_type not in valid_types:
        display(HTML(f"<b>Invalid</b> revenue_type: '<i>{revenue_type}</i>'. Must be one of <b>{valid_types}</b>"))

    plan = plans.get(row['plan'])
    if not plan:
        display(HTML(f"> <b>Invalid</b> plan: <i>{row['plan']}</i>, correct options: <b<('surf', 'ultimate')</b>"))

    # Calculate overages
    if revenue_type == 'duration':
        return max(0, row[column] - plan['minutes_limit']) * plan['minute_over']

    elif revenue_type == 'messages':
        return max(0, row[column] - plan['messages_limit']) * plan['message_over']

    elif revenue_type == 'traffic':
        return max(0, row[column] - plan['mb_limit']) * plan['mb_over']

    elif revenue_type == 'total':
        duration_excess = max(0, row[column[0]] - plan['minutes_limit']) * plan['minute_over']
        messages_excess = max(0, row[column[1]] - plan['messages_limit']) * plan['message_over']
        traffic_excess = max(0, row[column[2]] - plan['mb_limit']) * plan['mb_over']
        return plan['base_fee'] + duration_excess + messages_excess + traffic_excess


## 4 __Casting to data types__

### 3.1 Casting to string data type

In [5]:
# df_users 'first_name' to string
df_users['first_name'] = df_users['first_name'].astype('string')
df_users['first_name'].dtypes

string[python]

In [6]:
# df_users 'last_name' to string
df_users['last_name'] = df_users['last_name'].astype('string')
df_users['last_name'].dtypes

string[python]

In [7]:
# df_users 'city to string
df_users['city'] = df_users['city'].astype('string')
df_users['city'].dtypes

string[python]

### 3.2 Casting to category data type

In [8]:
# df_plans 'plan_name' to category
df_plans['plan_name'] = df_plans['plan_name'].astype('category')
df_plans['plan_name'].dtypes


CategoricalDtype(categories=['surf', 'ultimate'], ordered=False, categories_dtype=object)

In [9]:
# df_users 'plan' to category
df_users['plan'] = df_users['plan'].astype('category')
df_users['plan'].dtypes

CategoricalDtype(categories=['surf', 'ultimate'], ordered=False, categories_dtype=object)

### 3.3 Casting to datetime data type

In [10]:
# df_calls 'call_date' to datetime
df_calls = normalize_datetime(df_calls, include=['call_date'], frmt='%Y-%m-%d')
df_calls.dtypes

call_id                   object
user_id                    int64
call_date    datetime64[ns, UTC]
duration                 float64
dtype: object

In [11]:
# df_internet 'session_date' to datetime
df_internet = normalize_datetime(df_internet, include=['session_date'], frmt='%Y-%m-%d')
df_internet.dtypes

session_id                   object
user_id                       int64
session_date    datetime64[ns, UTC]
mb_used                     float64
dtype: object

In [12]:
# df_messages 'messages_date' to datetime
df_messages = normalize_datetime(df_messages, include=['message_date'], frmt='%Y-%m-%d')
df_messages.dtypes

message_id                   object
user_id                       int64
message_date    datetime64[ns, UTC]
dtype: object

In [13]:
# df_users 'reg_date' to datetime
df_users = normalize_datetime(df_users, include=['reg_date'], frmt='%Y-%m-%d')


In [14]:
# df_users 'churn_date' to datetime
df_users = normalize_datetime(df_users, include=['churn_date'], frmt='%Y-%m-%d')
df_users.dtypes

user_id                     int64
first_name         string[python]
last_name          string[python]
age                         int64
city               string[python]
reg_date      datetime64[ns, UTC]
plan                     category
churn_date    datetime64[ns, UTC]
dtype: object

## 4. Feature Engineering

### 4.1 Datasets

#### 4.1.1 Calls

In [15]:
df_calls

,call_id,user_id,call_date,duration
0,1000_93,1000,2018-12-27 00:00:00+00:00,8.52
1,1000_145,1000,2018-12-27 00:00:00+00:00,13.66
2,1000_247,1000,2018-12-27 00:00:00+00:00,14.48
3,1000_309,1000,2018-12-28 00:00:00+00:00,5.76
4,1000_380,1000,2018-12-30 00:00:00+00:00,4.22
...,...,...,...,...
137730,1499_199,1499,2018-11-21 00:00:00+00:00,8.72
137731,1499_200,1499,2018-10-20 00:00:00+00:00,10.89
137732,1499_201,1499,2018-09-21 00:00:00+00:00,8.12
137733,1499_202,1499,2018-10-10 00:00:00+00:00,0.37


In [16]:
# Each individual call is rounded up: even if the call lasted only one second
df_calls['duration'] = np.ceil(df_calls['duration'])

In [17]:
# Add Month
df_calls['month'] = df_calls['call_date'].dt.month

In [18]:
# Add Day
df_calls['day'] = df_calls['call_date'].dt.day

In [19]:
df_calls

,call_id,user_id,call_date,duration,month,day
0,1000_93,1000,2018-12-27 00:00:00+00:00,9.0,12,27
1,1000_145,1000,2018-12-27 00:00:00+00:00,14.0,12,27
2,1000_247,1000,2018-12-27 00:00:00+00:00,15.0,12,27
3,1000_309,1000,2018-12-28 00:00:00+00:00,6.0,12,28
4,1000_380,1000,2018-12-30 00:00:00+00:00,5.0,12,30
...,...,...,...,...,...,...
137730,1499_199,1499,2018-11-21 00:00:00+00:00,9.0,11,21
137731,1499_200,1499,2018-10-20 00:00:00+00:00,11.0,10,20
137732,1499_201,1499,2018-09-21 00:00:00+00:00,9.0,9,21
137733,1499_202,1499,2018-10-10 00:00:00+00:00,1.0,10,10


In [20]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "calls_feature.csv"

df_calls.to_csv(processed_path, index=False)

#### 4.1.2 Internet

In [21]:
df_internet

,session_id,user_id,session_date,mb_used
0,1000_13,1000,2018-12-29 00:00:00+00:00,89.86
1,1000_204,1000,2018-12-31 00:00:00+00:00,0.00
2,1000_379,1000,2018-12-28 00:00:00+00:00,660.40
3,1000_413,1000,2018-12-26 00:00:00+00:00,270.99
4,1000_442,1000,2018-12-27 00:00:00+00:00,880.22
...,...,...,...,...
104820,1499_215,1499,2018-10-20 00:00:00+00:00,218.06
104821,1499_216,1499,2018-12-30 00:00:00+00:00,304.72
104822,1499_217,1499,2018-09-22 00:00:00+00:00,292.75
104823,1499_218,1499,2018-12-07 00:00:00+00:00,0.00


In [22]:
# Add Month
df_internet['month'] = df_internet['session_date'].dt.month

In [23]:
# Add Day
df_internet['day'] = df_internet['session_date'].dt.day

In [24]:
df_internet

,session_id,user_id,session_date,mb_used,month,day
0,1000_13,1000,2018-12-29 00:00:00+00:00,89.86,12,29
1,1000_204,1000,2018-12-31 00:00:00+00:00,0.00,12,31
2,1000_379,1000,2018-12-28 00:00:00+00:00,660.40,12,28
3,1000_413,1000,2018-12-26 00:00:00+00:00,270.99,12,26
4,1000_442,1000,2018-12-27 00:00:00+00:00,880.22,12,27
...,...,...,...,...,...,...
104820,1499_215,1499,2018-10-20 00:00:00+00:00,218.06,10,20
104821,1499_216,1499,2018-12-30 00:00:00+00:00,304.72,12,30
104822,1499_217,1499,2018-09-22 00:00:00+00:00,292.75,9,22
104823,1499_218,1499,2018-12-07 00:00:00+00:00,0.00,12,7


In [25]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "internet_feature.csv"

df_internet.to_csv(processed_path, index=False)

#### 4.1.3 Messages

In [26]:
df_messages

,message_id,user_id,message_date
0,1000_125,1000,2018-12-27 00:00:00+00:00
1,1000_160,1000,2018-12-31 00:00:00+00:00
2,1000_223,1000,2018-12-31 00:00:00+00:00
3,1000_251,1000,2018-12-27 00:00:00+00:00
4,1000_255,1000,2018-12-26 00:00:00+00:00
...,...,...,...
76046,1497_526,1497,2018-12-24 00:00:00+00:00
76047,1497_536,1497,2018-12-24 00:00:00+00:00
76048,1497_547,1497,2018-12-31 00:00:00+00:00
76049,1497_558,1497,2018-12-24 00:00:00+00:00


In [27]:
# Add Month
df_messages['month'] = df_messages['message_date'].dt.month

In [28]:
# Add Day
df_messages['day'] = df_messages['message_date'].dt.day

In [29]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "messages_feature.csv"

df_messages.to_csv(processed_path, index=False)

#### 4.1.4 Users

In [30]:
df_users

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
0,1000,anamaria,bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",2018-12-24 00:00:00+00:00,ultimate,NaT
1,1001,mickey,wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",2018-08-13 00:00:00+00:00,surf,NaT
2,1002,carlee,hoffman,36,"las_vegas_henderson_paradise,_nv_msa",2018-10-21 00:00:00+00:00,surf,NaT
3,1003,reynaldo,jenkins,52,"tulsa,_ok_msa",2018-01-28 00:00:00+00:00,surf,NaT
4,1004,leonila,thompson,40,"seattle_tacoma_bellevue,_wa_msa",2018-05-23 00:00:00+00:00,surf,NaT
...,...,...,...,...,...,...,...,...
495,1495,fidel,sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",2018-09-04 00:00:00+00:00,surf,NaT
496,1496,ariel,shepherd,49,"new_orleans_metairie,_la_msa",2018-02-20 00:00:00+00:00,surf,NaT
497,1497,donte,barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",2018-12-10 00:00:00+00:00,ultimate,NaT
498,1498,scot,williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",2018-02-04 00:00:00+00:00,surf,NaT


In [31]:
# Full Name
df_users['full_name'] = df_users['first_name'] + "_" + df_users['last_name']

In [32]:
# Add Month 'reg_date'
df_users['reg_month'] = df_users['reg_date'].dt.month

In [33]:
# Add Day 'reg_date'
df_users['reg_day'] = df_users['reg_date'].dt.day

In [34]:
# Add Month 'churn_date'
df_users['churn_month'] = df_users['churn_date'].dt.month

In [35]:
# Add Day 'churn_date'
df_users['churn_day'] = df_users['churn_date'].dt.day

In [36]:
df_users

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date,full_name,reg_month,reg_day,churn_month,churn_day
0,1000,anamaria,bauer,45,"atlanta_sandy_springs_roswell,_ga_msa",2018-12-24 00:00:00+00:00,ultimate,NaT,anamaria_bauer,12,24,NaN,NaN
1,1001,mickey,wilkerson,28,"seattle_tacoma_bellevue,_wa_msa",2018-08-13 00:00:00+00:00,surf,NaT,mickey_wilkerson,8,13,NaN,NaN
2,1002,carlee,hoffman,36,"las_vegas_henderson_paradise,_nv_msa",2018-10-21 00:00:00+00:00,surf,NaT,carlee_hoffman,10,21,NaN,NaN
3,1003,reynaldo,jenkins,52,"tulsa,_ok_msa",2018-01-28 00:00:00+00:00,surf,NaT,reynaldo_jenkins,1,28,NaN,NaN
4,1004,leonila,thompson,40,"seattle_tacoma_bellevue,_wa_msa",2018-05-23 00:00:00+00:00,surf,NaT,leonila_thompson,5,23,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,fidel,sharpe,67,"new_york_newark_jersey_city,_ny_nj_pa_msa",2018-09-04 00:00:00+00:00,surf,NaT,fidel_sharpe,9,4,NaN,NaN
496,1496,ariel,shepherd,49,"new_orleans_metairie,_la_msa",2018-02-20 00:00:00+00:00,surf,NaT,ariel_shepherd,2,20,NaN,NaN
497,1497,donte,barrera,49,"los_angeles_long_beach_anaheim,_ca_msa",2018-12-10 00:00:00+00:00,ultimate,NaT,donte_barrera,12,10,NaN,NaN
498,1498,scot,williamson,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",2018-02-04 00:00:00+00:00,surf,NaT,scot_williamson,2,4,NaN,NaN


In [37]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "feature" / "users_feature.csv"

df_users.to_csv(processed_path, index=False)

### 4.2 Calls activity

In [38]:
df_calls

,call_id,user_id,call_date,duration,month,day
0,1000_93,1000,2018-12-27 00:00:00+00:00,9.0,12,27
1,1000_145,1000,2018-12-27 00:00:00+00:00,14.0,12,27
2,1000_247,1000,2018-12-27 00:00:00+00:00,15.0,12,27
3,1000_309,1000,2018-12-28 00:00:00+00:00,6.0,12,28
4,1000_380,1000,2018-12-30 00:00:00+00:00,5.0,12,30
...,...,...,...,...,...,...
137730,1499_199,1499,2018-11-21 00:00:00+00:00,9.0,11,21
137731,1499_200,1499,2018-10-20 00:00:00+00:00,11.0,10,20
137732,1499_201,1499,2018-09-21 00:00:00+00:00,9.0,9,21
137733,1499_202,1499,2018-10-10 00:00:00+00:00,1.0,10,10


#### 4.2.1 Calls amount and duration per user and month

In [39]:
df_calls_user_month = df_calls.groupby(['user_id', 'month'])['call_id'].count().reset_index()
df_calls_user_month = df_calls_user_month.rename(columns={'call_id': 'total_month_calls'})
df_calls_user_month

,user_id,month,total_month_calls
0,1000,12,16
1,1001,8,27
2,1001,9,49
3,1001,10,65
4,1001,11,64
...,...,...,...
2253,1498,12,39
2254,1499,9,41
2255,1499,10,53
2256,1499,11,45


In [40]:
df_duration_user_month = df_calls.groupby(['user_id', 'month'])['duration'].sum().reset_index(drop=True)
df_duration_user_month.name = 'total_month_duration'
df_duration_user_month

0       124.0
1       182.0
2       315.0
3       393.0
4       426.0
        ...  
2253    339.0
2254    346.0
2255    385.0
2256    308.0
2257    496.0
Name: total_month_duration, Length: 2258, dtype: float64

In [41]:
df_calls_duration_month = pd.concat([df_calls_user_month, df_duration_user_month], axis=1)
df_calls_duration_month                                    
                                    

,user_id,month,total_month_calls,total_month_duration
0,1000,12,16,124.0
1,1001,8,27,182.0
2,1001,9,49,315.0
3,1001,10,65,393.0
4,1001,11,64,426.0
...,...,...,...,...
2253,1498,12,39,339.0
2254,1499,9,41,346.0
2255,1499,10,53,385.0
2256,1499,11,45,308.0


### 4.3 Messages activity

In [42]:
df_messages

,message_id,user_id,message_date,month,day
0,1000_125,1000,2018-12-27 00:00:00+00:00,12,27
1,1000_160,1000,2018-12-31 00:00:00+00:00,12,31
2,1000_223,1000,2018-12-31 00:00:00+00:00,12,31
3,1000_251,1000,2018-12-27 00:00:00+00:00,12,27
4,1000_255,1000,2018-12-26 00:00:00+00:00,12,26
...,...,...,...,...,...
76046,1497_526,1497,2018-12-24 00:00:00+00:00,12,24
76047,1497_536,1497,2018-12-24 00:00:00+00:00,12,24
76048,1497_547,1497,2018-12-31 00:00:00+00:00,12,31
76049,1497_558,1497,2018-12-24 00:00:00+00:00,12,24


#### 4.3.1 Messages amount per user and month

In [43]:
df_messages_user_month = df_messages.groupby(['user_id', 'month'])['message_id'].count().reset_index()
df_messages_user_month = df_messages_user_month.rename(columns={'message_id': 'total_month_messages'})
df_messages_user_month

,user_id,month,total_month_messages
0,1000,12,11
1,1001,8,30
2,1001,9,44
3,1001,10,53
4,1001,11,36
...,...,...,...
1801,1496,9,21
1802,1496,10,18
1803,1496,11,13
1804,1496,12,11


### 4.4 Internet activity

In [44]:
df_internet

,session_id,user_id,session_date,mb_used,month,day
0,1000_13,1000,2018-12-29 00:00:00+00:00,89.86,12,29
1,1000_204,1000,2018-12-31 00:00:00+00:00,0.00,12,31
2,1000_379,1000,2018-12-28 00:00:00+00:00,660.40,12,28
3,1000_413,1000,2018-12-26 00:00:00+00:00,270.99,12,26
4,1000_442,1000,2018-12-27 00:00:00+00:00,880.22,12,27
...,...,...,...,...,...,...
104820,1499_215,1499,2018-10-20 00:00:00+00:00,218.06,10,20
104821,1499_216,1499,2018-12-30 00:00:00+00:00,304.72,12,30
104822,1499_217,1499,2018-09-22 00:00:00+00:00,292.75,9,22
104823,1499_218,1499,2018-12-07 00:00:00+00:00,0.00,12,7


#### 4.4.1 Internet sessions and traffic per user and month

In [45]:
df_sessions_user_month = df_internet.groupby(['user_id', 'month'])['session_id'].count().reset_index()
df_sessions_user_month = df_sessions_user_month.rename(columns={'session_id': 'total_month_sessions'})
df_sessions_user_month

,user_id,month,total_month_sessions
0,1000,12,5
1,1001,8,25
2,1001,9,53
3,1001,10,56
4,1001,11,51
...,...,...,...
2272,1498,12,65
2273,1499,9,45
2274,1499,10,61
2275,1499,11,49


In [46]:
# Individual web sessions are not rounded up. Instead, the monthly total is rounded up
df_traffic_user_month = df_internet.groupby(['user_id', 'month'])['mb_used'].sum().reset_index()
df_traffic_user_month = df_traffic_user_month.rename(columns={'mb_used': 'total_month_traffic'})
df_traffic_user_month['total_month_traffic'] = np.ceil(df_traffic_user_month['total_month_traffic'] / 1000) * 1000
df_traffic_user_month = df_traffic_user_month.drop(['user_id', 'month'], axis=1)
df_traffic_user_month

,total_month_traffic
0,2000.0
1,7000.0
2,14000.0
3,23000.0
4,19000.0
...,...
2272,24000.0
2273,13000.0
2274,20000.0
2275,17000.0


In [47]:
df_sessions_traffic_month = pd.concat([df_sessions_user_month, df_traffic_user_month], axis=1)
df_sessions_traffic_month

,user_id,month,total_month_sessions,total_month_traffic
0,1000,12,5,2000.0
1,1001,8,25,7000.0
2,1001,9,53,14000.0
3,1001,10,56,23000.0
4,1001,11,51,19000.0
...,...,...,...,...
2272,1498,12,65,24000.0
2273,1499,9,45,13000.0
2274,1499,10,61,20000.0
2275,1499,11,49,17000.0


### 4.5 Services activity per month and day

#### 4.5.1 Calls per month and day

In [48]:
df_calls_duration_periodic_day = df_calls.groupby(['month', 'day'])['duration'].sum().reset_index()
df_calls_duration_periodic_day = df_calls_duration_periodic_day.rename(columns={'duration': 'calls_duration'})
df_calls_duration_periodic_day

,month,day,calls_duration
0,1,15,32.0
1,1,16,73.0
2,1,17,29.0
3,1,18,53.0
4,1,19,76.0
...,...,...,...
346,12,27,7906.0
347,12,28,7970.0
348,12,29,7770.0
349,12,30,7641.0


In [63]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "activity" / "calls_duration_periodic_day.csv"

df_calls_duration_periodic_day.to_csv(processed_path, index=False)

#### 4.5.2 Messages per month and day

In [50]:
df_messages_periodic_day = df_messages.groupby(['month', 'day'])['message_id'].count().reset_index()
df_messages_periodic_day = df_messages_periodic_day.rename(columns={'message_id': 'message_count'})
df_messages_periodic_day

,month,day,message_count
0,1,15,2
1,1,16,4
2,1,17,6
3,1,18,5
4,1,19,1
...,...,...,...
346,12,27,701
347,12,28,702
348,12,29,680
349,12,30,683


In [51]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "activity" / "messages_periodic_day.csv"

df_messages_periodic_day.to_csv(processed_path, index=False)

#### 4.5.3 Internet per month and day

In [52]:
df_internet_periodic_day = df_internet.groupby(['month', 'day'])['mb_used'].sum().reset_index()
df_internet_periodic_day = df_internet_periodic_day.rename(columns={'mb_used': 'traffic_usage'})
df_internet_periodic_day

,month,day,traffic_usage
0,1,15,1193.90
1,1,16,1776.09
2,1,17,1202.33
3,1,18,931.02
4,1,19,2684.85
...,...,...,...
346,12,27,291749.27
347,12,28,291703.03
348,12,29,288687.21
349,12,30,324013.99


In [53]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "activity" / "internet_periodic_day.csv"

df_internet_periodic_day.to_csv(processed_path, index=False)

### 4.6 Revenue

#### 4.6.1 Monthly evenue per user

In [54]:
df_revenue_month = df_calls_duration_month.copy()
df_revenue_month

,user_id,month,total_month_calls,total_month_duration
0,1000,12,16,124.0
1,1001,8,27,182.0
2,1001,9,49,315.0
3,1001,10,65,393.0
4,1001,11,64,426.0
...,...,...,...,...
2253,1498,12,39,339.0
2254,1499,9,41,346.0
2255,1499,10,53,385.0
2256,1499,11,45,308.0


In [55]:
df_revenue_month = df_revenue_month.merge(df_messages_user_month, on=['user_id', 'month'], how='outer')
df_revenue_month

,user_id,month,total_month_calls,total_month_duration,total_month_messages
0,1000,12,16.0,124.0,11.0
1,1001,8,27.0,182.0,30.0
2,1001,9,49.0,315.0,44.0
3,1001,10,65.0,393.0,53.0
4,1001,11,64.0,426.0,36.0
...,...,...,...,...,...
2286,1498,12,39.0,339.0,NaN
2287,1499,9,41.0,346.0,NaN
2288,1499,10,53.0,385.0,NaN
2289,1499,11,45.0,308.0,NaN


In [56]:
df_revenue_month = df_revenue_month.merge(df_sessions_traffic_month, on=['user_id', 'month'], how='outer')
df_revenue_month

,user_id,month,total_month_calls,total_month_duration,total_month_messages,total_month_sessions,total_month_traffic
0,1000,12,16.0,124.0,11.0,5.0,2000.0
1,1001,8,27.0,182.0,30.0,25.0,7000.0
2,1001,9,49.0,315.0,44.0,53.0,14000.0
3,1001,10,65.0,393.0,53.0,56.0,23000.0
4,1001,11,64.0,426.0,36.0,51.0,19000.0
...,...,...,...,...,...,...,...
2288,1498,12,39.0,339.0,NaN,65.0,24000.0
2289,1499,9,41.0,346.0,NaN,45.0,13000.0
2290,1499,10,53.0,385.0,NaN,61.0,20000.0
2291,1499,11,45.0,308.0,NaN,49.0,17000.0


In [57]:
df_revenue_month = df_revenue_month.merge(df_users[['user_id', 'age', 'city', 'plan', 'full_name', 'churn_month']], on='user_id', how='inner')
df_revenue_month

,user_id,month,total_month_calls,total_month_duration,total_month_messages,total_month_sessions,total_month_traffic,age,city,plan,full_name,churn_month
0,1000,12,16.0,124.0,11.0,5.0,2000.0,45,"atlanta_sandy_springs_roswell,_ga_msa",ultimate,anamaria_bauer,NaN
1,1001,8,27.0,182.0,30.0,25.0,7000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN
2,1001,9,49.0,315.0,44.0,53.0,14000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN
3,1001,10,65.0,393.0,53.0,56.0,23000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN
4,1001,11,64.0,426.0,36.0,51.0,19000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2288,1498,12,39.0,339.0,NaN,65.0,24000.0,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",surf,scot_williamson,NaN
2289,1499,9,41.0,346.0,NaN,45.0,13000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN
2290,1499,10,53.0,385.0,NaN,61.0,20000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN
2291,1499,11,45.0,308.0,NaN,49.0,17000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN


In [58]:
df_revenue_month['month_revenue_calls'] = np.round(df_revenue_month.apply(lambda row: revenue(row, 'duration', 'total_month_duration'), axis=1), 3)
df_revenue_month

,user_id,month,total_month_calls,total_month_duration,total_month_messages,total_month_sessions,total_month_traffic,age,city,plan,full_name,churn_month,month_revenue_calls
0,1000,12,16.0,124.0,11.0,5.0,2000.0,45,"atlanta_sandy_springs_roswell,_ga_msa",ultimate,anamaria_bauer,NaN,0.0
1,1001,8,27.0,182.0,30.0,25.0,7000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0
2,1001,9,49.0,315.0,44.0,53.0,14000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0
3,1001,10,65.0,393.0,53.0,56.0,23000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0
4,1001,11,64.0,426.0,36.0,51.0,19000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288,1498,12,39.0,339.0,NaN,65.0,24000.0,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",surf,scot_williamson,NaN,0.0
2289,1499,9,41.0,346.0,NaN,45.0,13000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0
2290,1499,10,53.0,385.0,NaN,61.0,20000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0
2291,1499,11,45.0,308.0,NaN,49.0,17000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0


In [59]:
df_revenue_month['month_revenue_messages'] = np.round(df_revenue_month.apply(lambda row: revenue(row, 'messages', 'total_month_messages'), axis=1), 3)
df_revenue_month

,user_id,month,total_month_calls,total_month_duration,total_month_messages,total_month_sessions,total_month_traffic,age,city,plan,full_name,churn_month,month_revenue_calls,month_revenue_messages
0,1000,12,16.0,124.0,11.0,5.0,2000.0,45,"atlanta_sandy_springs_roswell,_ga_msa",ultimate,anamaria_bauer,NaN,0.0,0.00
1,1001,8,27.0,182.0,30.0,25.0,7000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.00
2,1001,9,49.0,315.0,44.0,53.0,14000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.00
3,1001,10,65.0,393.0,53.0,56.0,23000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.09
4,1001,11,64.0,426.0,36.0,51.0,19000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288,1498,12,39.0,339.0,NaN,65.0,24000.0,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",surf,scot_williamson,NaN,0.0,0.00
2289,1499,9,41.0,346.0,NaN,45.0,13000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0,0.00
2290,1499,10,53.0,385.0,NaN,61.0,20000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0,0.00
2291,1499,11,45.0,308.0,NaN,49.0,17000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0,0.00


In [60]:
df_revenue_month['month_revenue_internet'] = np.round(df_revenue_month.apply(lambda row: revenue(row, 'traffic', 'total_month_traffic'), axis=1), 3)
df_revenue_month

,user_id,month,total_month_calls,total_month_duration,total_month_messages,total_month_sessions,total_month_traffic,age,city,plan,full_name,churn_month,month_revenue_calls,month_revenue_messages,month_revenue_internet
0,1000,12,16.0,124.0,11.0,5.0,2000.0,45,"atlanta_sandy_springs_roswell,_ga_msa",ultimate,anamaria_bauer,NaN,0.0,0.00,0.0
1,1001,8,27.0,182.0,30.0,25.0,7000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.00,0.0
2,1001,9,49.0,315.0,44.0,53.0,14000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.00,0.0
3,1001,10,65.0,393.0,53.0,56.0,23000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.09,76.4
4,1001,11,64.0,426.0,36.0,51.0,19000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.00,36.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288,1498,12,39.0,339.0,NaN,65.0,24000.0,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",surf,scot_williamson,NaN,0.0,0.00,86.4
2289,1499,9,41.0,346.0,NaN,45.0,13000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0,0.00,0.0
2290,1499,10,53.0,385.0,NaN,61.0,20000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0,0.00,46.4
2291,1499,11,45.0,308.0,NaN,49.0,17000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0,0.00,16.4


In [61]:
df_revenue_month['month_revenue_total'] = np.round(df_revenue_month.apply(lambda row: revenue(row, 'total', ['total_month_duration', 'total_month_messages', 'total_month_traffic']), axis=1), 3)
df_revenue_month

,user_id,month,total_month_calls,total_month_duration,total_month_messages,total_month_sessions,total_month_traffic,age,city,plan,full_name,churn_month,month_revenue_calls,month_revenue_messages,month_revenue_internet,month_revenue_total
0,1000,12,16.0,124.0,11.0,5.0,2000.0,45,"atlanta_sandy_springs_roswell,_ga_msa",ultimate,anamaria_bauer,NaN,0.0,0.00,0.0,70.00
1,1001,8,27.0,182.0,30.0,25.0,7000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.00,0.0,20.00
2,1001,9,49.0,315.0,44.0,53.0,14000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.00,0.0,20.00
3,1001,10,65.0,393.0,53.0,56.0,23000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.09,76.4,96.49
4,1001,11,64.0,426.0,36.0,51.0,19000.0,28,"seattle_tacoma_bellevue,_wa_msa",surf,mickey_wilkerson,NaN,0.0,0.00,36.4,56.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288,1498,12,39.0,339.0,NaN,65.0,24000.0,51,"new_york_newark_jersey_city,_ny_nj_pa_msa",surf,scot_williamson,NaN,0.0,0.00,86.4,106.40
2289,1499,9,41.0,346.0,NaN,45.0,13000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0,0.00,0.0,20.00
2290,1499,10,53.0,385.0,NaN,61.0,20000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0,0.00,46.4,66.40
2291,1499,11,45.0,308.0,NaN,49.0,17000.0,37,"orlando_kissimmee_sanford,_fl_msa",surf,shena_dickson,NaN,0.0,0.00,16.4,36.40


In [62]:
project_root = Path.cwd().parent
processed_path = project_root / "data" / "processed" / "activity" / "revenue_total.csv"

df_revenue_month.to_csv(processed_path, index=False)